In [1]:
import numpy as np
import pandas as pd
import os
import time
import json

import warnings
warnings.filterwarnings('ignore')

In [53]:
twitter_path = 'data/Twitter_trainingandtestdata/'
football_path = 'data/FootBall_archive/dataset/'
pubmed_path = 'data/PubMedQA/'
squad_path = 'data/SQuAD/'
novels_path = 'data/Project Gutenberg/'

In [34]:
main_df = pd.read_csv("unprocessed36538Data.csv")

In [35]:
main_df

,sentence,class
0,Partay from 45 yards out! Unbelievable! Great ...,1
1,Albert Montañés was a legendary player in the ...,1
2,"This team had potential, but without a goal to...",1
3,"The team, once part of a national championship...",1
4,And there you have it folks! Being number one ...,1
...,...,...
36533,We're looking at a potential penalty here as t...,1
36534,"And he throws the pigskin forward, down to the...",1
36535,"Fiction's Giants on Pittsburg's 17-yard line, ...",1
36536,Number 71 from Pittsburgh shoots! It's a despe...,1


In [37]:
twitter_df = pd.read_csv(twitter_path + 'training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1", header = None)
twitter_df.rename(columns = {0:"sentiments",1:"ids",2:"DateTime",3:"NO_QUERY",4:"UserId",5:"review"}, inplace = True)
twitter_df

,sentiments,ids,DateTime,NO_QUERY,UserId,review
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [38]:
football_df =  pd.read_json(football_path + 'tagged_transcripts.json').T
football_df

,teams,transcript,year
1962-houston_oilers-dallas_texans.txt,"[houston_oilers, dallas_texans]",gilson well defend the goal on your left theyl...,1962
1969-chicago_bears-green_bay_packers.txt,"[chicago_bears, green_bay_packers]",cbs television sports presents the national fo...,1969
1969-cleveland_browns-minnesota_vikings-1.txt,"[cleveland_browns, minnesota_vikings]",the nfl today brought to you by the foundation...,1969
1969-cleveland_browns-minnesota_vikings.txt,"[cleveland_browns, minnesota_vikings]",the nfl today brought to you by the foundation...,1969
1969-new_york_jets-baltimore_colts.txt,"[new_york_jets, baltimore_colts]",&gt;&gt; nbc sports presents the third nflafl ...,1969
...,...,...,...
2018-tennessee_titans-tampa_bay_buccaneers.txt,"[tennessee_titans, tampa_bay_buccaneers]",tennessee titans preseason football is brought...,2018
2018-washington_redskins-new_england_patriots.txt,"[washington_redskins, new_england_patriots]",the patriots take the field and football retur...,2018
2018-washington_redskins-new_york_jets.txt,"[washington_redskins, new_york_jets]",espn welcomes you to the following presentatio...,2018
2018-washington_redskins-philadelphia_eagles-1.txt,"[washington_redskins, philadelphia_eagles]",and there is nick falls the eagle fans at atte...,2018


In [40]:
pubmed_ans_df = pd.read_json(pubmed_path + 'ori_pqaa.json').T
pubmed_ans_df

,QUESTION,CONTEXTS,LABELS,LONG_ANSWER,MESHES,final_decision
25429730,Are group 2 innate lymphoid cells ( ILC2s ) in...,[Chronic rhinosinusitis (CRS) is a heterogeneo...,"[BACKGROUND, OBJECTIVE, METHODS, RESULTS]","As ILC2s are elevated in patients with CRSwNP,...","[Adult, Aged, Antigens, Surface, Case-Control ...",yes
25433161,Does vagus nerve contribute to the development...,[Phosphatidylethanolamine N-methyltransferase ...,"[OBJECTIVE, METHODS, RESULTS]",Neuronal signals via the hepatic vagus nerve c...,"[Animals, Chemokine CCL2, Diet, High-Fat, Dise...",yes
25445714,Does psammaplin A induce Sirtuin 1-dependent a...,[Psammaplin A (PsA) is a natural product isola...,"[BACKGROUND, METHODS]",PsA significantly inhibited MCF-7/adr cells pr...,"[Acetylation, Animals, Antibiotics, Antineopla...",yes
25431941,Is methylation of the FGFR2 gene associated wi...,[This study examined links between DNA methyla...,"[OBJECTIVE, METHODS, RESULTS]",We identified a novel biologically plausible c...,[],yes
25432519,Do tumor-infiltrating immune cell profiles and...,[Tumor microenvironment immunity is associated...,"[BACKGROUND, METHODS, RESULTS]",Breast cancer immune cell subpopulation profil...,"[Adult, Aged, Anthracyclines, Antibodies, Mono...",yes
...,...,...,...,...,...,...
8217974,Is urine production rate related to behavioura...,[To investigate the relation between hourly fe...,"[OBJECTIVE, METHODS, METHODS, METHODS, METHODS...",During active sleep (state 2F) hourly fetal ur...,"[Behavior, Embryonic and Fetal Development, Fe...",yes
8204319,Does evaluation of the use of general practice...,[This study set out to show how well samples f...,"[OBJECTIVE, METHODS, RESULTS]",General practice registers can provide a suita...,"[Adult, Age Factors, Epidemiologic Methods, Fa...",yes
8205673,Does intracoronary angiotensin-converting enzy...,[There is increasing recognition of myocardial...,"[BACKGROUND, RESULTS]",Intracoronary enalaprilat resulted in an impro...,"[Adult, Aged, Coronary Vessels, Diastole, Enal...",yes
8215873,Does transfusion significantly increase the ri...,[To determine if splenectomy results in an inc...,"[OBJECTIVE, METHODS, METHODS, METHODS, METHODS...",The choice between splenectomy and splenic rep...,"[Adult, Bacteremia, Female, Humans, Injury Sev...",yes


In [41]:
pubmed_ques_df = pd.read_json(pubmed_path + 'ori_pqau.json').T
pubmed_ques_df

,QUESTION,CONTEXTS,LABELS,MESHES,YEAR,LONG_ANSWER
14499029,Is naturopathy as effective as conventional th...,[Although the use of alternative medicine in t...,"[BACKGROUND, OBJECTIVE, DESIGN, SETTING, PATIE...","[Anxiety, Cohort Studies, Confidence Intervals...",2003,Naturopathy appears to be an effective alterna...
14499049,Can randomised trials rely on existing electro...,"[To estimate the feasibility, utility and reso...","[OBJECTIVES, DATA SOURCES, REVIEW METHODS, RES...","[Arthroplasty, Replacement, Knee, Bias, Blood ...",2003,Routine data have the potential to support hea...
14499672,Is laparoscopic radical prostatectomy better t...,[To compare morbidity in two groups of patient...,"[OBJECTIVE, PATIENTS AND METHODS, RESULTS]","[Aged, Follow-Up Studies, Humans, Italy, Lapar...",2003,The results of our non-randomized study show t...
14499773,Does bacterial gastroenteritis predispose peop...,[Irritable bowel syndrome (IBS) might develop ...,"[OBJECTIVES, METHODS, RESULTS]","[Adolescent, Adult, Age Distribution, Aged, Ag...",2003,Symptoms consistent with IBS and functional di...
14499777,Is early colonoscopy after admission for acute...,[Urgent colonoscopy has been proposed for the ...,"[OBJECTIVES, METHODS, RESULTS]","[Acute Disease, Aged, Aged, 80 and over, Cohor...",2003,No significant association is apparent between...
...,...,...,...,...,...,...
10632750,Diversion colitis in children: an iatrogenic a...,"[Diversion colitis (DC) is a localized, relati...","[AIMS, METHODS AND RESULTS]","[Appendix, Child, Preschool, Colitis, Colitis,...",2000,Histological features of DC in children are ve...
10632796,Raising research awareness among midwives and ...,[The primary aim of the study was to evaluate ...,"[OBJECTIVE, DESIGN, SAMPLE, ETHICS, DATA COLLE...","[Attitude of Health Personnel, Chi-Square Dist...",2000,The introduction of clinical governance challe...
10632828,Delivery of primary care to women. Do women's ...,[Women's health centers have been increasing i...,"[OBJECTIVE, DESIGN, SETTING, PARTICIPANTS, MEA...","[Adolescent, Adult, Aged, Cross-Sectional Stud...",2000,"These results suggest that, at least in this s..."
10633786,Analysis of failures after whole abdominal irr...,[To evaluate failures and to investigate the n...,"[BACKGROUND, PATIENTS AND METHOD, RESULTS]","[Abdomen, Adult, Aged, Aged, 80 and over, Comb...",1999,General prophylactic enclosure of the inguinal...


In [43]:
squad_train = json.load(open(squad_path + "train-v2.0.json"))
squad_dev = json.load(open(squad_path + "dev-v2.0.json"))

In [48]:
parsed_squad = []

for k in squad_train['data']:
    for i in range(len(k['paragraphs'])):
        p = k['paragraphs'][i]
             

        for j in range(len(p['qas'])):
            v = p['qas'][j]
            
            row = {}
            row['title'] = k['title']
            row['context'] = p['context']
            row['question'] = v['question']
            row['id'] = v['id']
            
            try:
                row['answers'] = v['answers'][0]['text']
                row['answer_start'] = v['answers'][0]['answer_start']
            except Exception as e:
                row['answers'] = ""
                row['answer_start'] = ""
            
            row['is_impossible'] = v['is_impossible']
            parsed_squad.append(row)

            
for k in squad_dev['data']:
    for i in range(len(k['paragraphs'])):
        p = k['paragraphs'][i]
             

        for j in range(len(p['qas'])):
            v = p['qas'][j]
            
            row = {}
            row['title'] = k['title']
            row['context'] = p['context']
            row['question'] = v['question']
            row['id'] = v['id']
            
            try:
                row['answers'] = v['answers'][0]['text']
                row['answer_start'] = v['answers'][0]['answer_start']
            except Exception as e:
                row['answers'] = ""
                row['answer_start'] = ""
            
            row['is_impossible'] = v['is_impossible']
            parsed_squad.append(row)


In [49]:
squad_train_df = pd.DataFrame.from_dict(parsed_squad)
squad_train_df

,title,context,question,id,answers,answer_start,is_impossible
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,in the late 1990s,269,False
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,singing and dancing,207,False
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,2003,526,False
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,"Houston, Texas",166,False
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,late 1990s,276,False
...,...,...,...,...,...,...,...
142187,Force,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,5737aafd1c456719005744ff,sthène,665,False
142188,Force,"The pound-force has a metric counterpart, less...",What does not have a metric counterpart?,5ad28ad0d7d075001a4299cc,,,True
142189,Force,"The pound-force has a metric counterpart, less...",What is the force exerted by standard gravity ...,5ad28ad0d7d075001a4299cd,,,True
142190,Force,"The pound-force has a metric counterpart, less...",What force leads to a commonly used unit of mass?,5ad28ad0d7d075001a4299ce,,,True


In [51]:
os.listdir(novels_path)

['Adventures_of_Huckleberry_Finn_by_Mark_Twain.rtf',
 'Alices_Adventures_in_Wonderland_by_Lewis_Carroll.rtf',
 'A_Dolls_House_by_Henrik_Ibsen.rtf',
 'A_Tale_of_Two_Cities_by_Charles_Dickens.rtf',
 'Dracula_by_Bram_Stoker.rtf',
 'Emma_by_Jane_Austen.rtf',
 'Frankenstein_by_Mary_Shelley.rtf',
 'Great_Expectations_by_Charles_Dickens.rtf',
 'Grimms_Fairy_Tales_by_The_Brothers_Grimm.rtf',
 'Metamorphosis_by_Franz_Kafka.rtf',
 'Pride_and_Prejudice_by_Jane_Austen.rtf',
 'project_gutenberg_archive.zip',
 'The_Adventures_of_Sherlock_Holmes_by_Arthur_Conan_Doyle.rtf',
 'The_Adventures_of_Tom_Sawyer_by_Mark_Twain.rtf',
 'The_Count_of_Monte_Cristo_by_Alexandre_Dumas.rtf',
 'The_Importance_of_Being_Earnest_by_Oscar_Wilde.rtf',
 'The_Picture_of_Dorian_Gray_by_Oscar_Wilde.rtf',
 'The_Prince_by_Nicolo_Machiavelli.rtf',
 'The_Romance_of_Lust_by_Anonymous.rtf',
 'The_Yellow_Wallpaper_by_Charlotte_Perkins_Gilman.rtf',
 'Ulysses_by_James_Joyce.rtf']

In [62]:
from striprtf.striprtf import rtf_to_text
novels_data = ""
with open(novels_path + 'Adventures_of_Huckleberry_Finn_by_Mark_Twain.rtf') as infile:
    content = infile.read()
    novels_data = rtf_to_text(content)
print(novels_data)



The Project Gutenberg EBook of Adventures of Huckleberry Finn, Complete
by Mark Twain (Samuel Clemens)

This eBook is for the use of anyone anywhere at no cost and with almost
no restrictions whatsoever. You may copy it, give it away or re-use
it under the terms of the Project Gutenberg License included with this
eBook or online at www.gutenberg.net

Title: Adventures of Huckleberry Finn, Complete

Author: Mark Twain (Samuel Clemens)

Release Date: August 20, 2006 [EBook #76]

Last Updated: August 17, 2016]

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK HUCKLEBERRY FINN ***

Produced by David Widger





ADVENTURES

OF

HUCKLEBERRY FINN

(Tom Sawyer’s Comrade)

By Mark Twain

Complete




CONTENTS.

CHAPTER I. Civilizing Huck.--Miss Watson.--Tom Sawyer Waits.

CHAPTER II. The Boys Escape Jim.--Torn Sawyer’s Gang.--Deep-laid Plans.

CHAPTER III. A Good Going-over.--Grace Triumphant.--“One of Tom Sawyers’s
Lies”.

CHAPTER IV. Huck and the J

In [68]:
novels_data_list = novels_data.split("\n")
novel_sentences = []
start_append = False
for text in novels_data_list:
    if text == "CHAPTER I.":
        start_append = True
    if start_append:
        if text[:7] != "CHAPTER" and text != "":
            novel_sentences.append(text)

In [76]:
novel_sentences

['YOU don’t know about me without you have read a book by the name of The',
 'Adventures of Tom Sawyer; but that ain’t no matter. \xa0That book was made',
 'by Mr. Mark Twain, and he told the truth, mainly. \xa0There was things',
 'which he stretched, but mainly he told the truth. \xa0That is nothing. \xa0I',
 'never seen anybody but lied one time or another, without it was Aunt',
 'Polly, or the widow, or maybe Mary. \xa0Aunt Polly--Tom’s Aunt Polly, she',
 'is--and Mary, and the Widow Douglas is all told about in that book, which',
 'is mostly a true book, with some stretchers, as I said before.',
 'Now the way that the book winds up is this: \xa0Tom and me found the money',
 'that the robbers hid in the cave, and it made us rich. \xa0We got six',
 'thousand dollars apiece--all gold. \xa0It was an awful sight of money when',
 'it was piled up. \xa0Well, Judge Thatcher he took it and put it out',
 'at interest, and it fetched us a dollar a day apiece all the year',
 'round--more than 

In [78]:
novels_data2 = ""
with open(novels_path + 'Grimms_Fairy_Tales_by_The_Brothers_Grimm.rtf') as infile:
    content = infile.read()
    novels_data2 = rtf_to_text(content)
print(novels_data2)

The Project Gutenberg EBook of Grimms’ Fairy Tales, by The Brothers Grimm

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Grimms’ Fairy Tales

Author: The Brothers Grimm

Translator: Edgar Taylor and Marian Edwardes

Posting Date: December 14, 2008 [EBook #2591]
Release Date: April, 2001
Last Updated: November 7, 2016

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK GRIMMS’ FAIRY TALES ***




Produced by Emma Dudding, John Bickers, and Dagny





FAIRY TALES

By The Brothers Grimm



PREPARER’S NOTE

     The text is based on translations from
     the Grimms’ Kinder und Hausmarchen by
     Edgar Taylor and Marian Edwardes.




CONTENTS:

     THE GOLDEN BIRD
     HANS IN LUCK
     JORINDA AND JORINDEL
     THE TRAVELLING MUSICIANS
  